<a href="https://colab.research.google.com/github/EAName/yelp-reviews/blob/main/Yelp_Data_Converting_Review_Json_to_CSV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import json
import os

import numpy as np
import pandas as pd
from tqdm import tqdm

Business file

In [3]:
# First drop yelp_dataset into your drive (Add shortcut to Drive)
business_json_path = '/content/gdrive/MyDrive/yelp_dataset/yelp_academic_dataset_business.json'
df_b = pd.read_json(business_json_path, lines=True)

In [4]:
df_b.size

2104844

In [5]:
# #Drop closed businesses

# # 1 = open, 0 = closed
# df_b = df_b[df_b['is_open']==1]

In [6]:
#keep only restaurant category
df_b = df_b[df_b['categories'].str.contains(
              'restaurant|Restaurant|RESTAURANT',
              case=False, na=False)]

In [7]:
df_b.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
5,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,615 S Main St,Ashland City,TN,37015,36.269593,-87.058943,2.0,6,1,"{'BusinessParking': 'None', 'BusinessAcceptsCr...","Burgers, Fast Food, Sandwiches, Food, Ice Crea...","{'Monday': '0:0-0:0', 'Tuesday': '6:0-22:0', '..."
8,k0hlBqXX-Bt0vf1op7Jr1w,Tsevi's Pub And Grill,8025 Mackenzie Rd,Affton,MO,63123,38.565165,-90.321087,3.0,19,0,"{'Caters': 'True', 'Alcohol': 'u'full_bar'', '...","Pubs, Restaurants, Italian, Bars, American (Tr...",None
9,bBDDEgkFA1Otx9Lfe7BZUQ,Sonic Drive-In,2312 Dickerson Pike,Nashville,TN,37207,36.208102,-86.768170,1.5,10,1,"{'RestaurantsAttire': ''casual'', 'Restaurants...","Ice Cream & Frozen Yogurt, Fast Food, Burgers,...","{'Monday': '0:0-0:0', 'Tuesday': '6:0-21:0', '..."
11,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,,Tampa Bay,FL,33602,27.955269,-82.456320,4.0,10,1,"{'Alcohol': ''none'', 'OutdoorSeating': 'None'...","Vietnamese, Food, Restaurants, Food Trucks","{'Monday': '11:0-14:0', 'Tuesday': '11:0-14:0'..."


In [8]:
df_explode = df_b.assign(categories = df_b.categories
                         .str.split(', ')).explode('categories')

In [9]:
df_explode.categories.value_counts()

Restaurants                    52268
Food                           15477
Nightlife                       8723
Sandwiches                      8366
Bars                            8337
                               ...  
Check Cashing/Pay-day Loans        1
Kiosk                              1
Roman                              1
Pressure Washers                   1
Gemstones & Minerals               1
Name: categories, Length: 730, dtype: int64

Review file

In [10]:
review_json_path = '/content/gdrive/MyDrive/yelp_dataset/yelp_academic_dataset_review.json'

In [11]:
data = {'user_id': [], 'business_id': [], 'date': [], 'stars': [], 'text': []}

with open('/content/gdrive/MyDrive/yelp_dataset/yelp_academic_dataset_review.json') as f:
    for line in tqdm(f):
        review = json.loads(line)
        data['user_id'].append(review['user_id'])
        data['business_id'].append(review['business_id'])
        data['date'].append(review['date'])
        data['stars'].append(review['stars'])
        data['text'].append(review['text'])

6990280it [02:43, 42629.77it/s]


In [12]:
df = pd.DataFrame(data)

print(df.shape)
df.head()

(6990280, 5)


,user_id,business_id,date,stars,text
0,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,2018-07-07 22:09:11,3.0,"If you decide to eat here, just be aware it is..."
1,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,2012-01-03 15:28:18,5.0,I've taken a lot of spin classes over the year...
2,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,2014-02-05 20:30:30,3.0,Family diner. Had the buffet. Eclectic assortm...
3,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,2015-01-04 00:01:03,5.0,"Wow! Yummy, different, delicious. Our favo..."
4,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,2017-01-14 20:54:15,4.0,Cute interior and owner (?) gave us tour of up...


In [14]:
# only keep businesses that are categorized as restaurant
df2 = df.merge(df_b,left_on='business_id',right_on='business_id')

In [15]:
print(df2.shape)

(4724684, 18)


In [16]:
df2.head()

,user_id,business_id,date,stars_x,text,name,address,city,state,postal_code,latitude,longitude,stars_y,review_count,is_open,attributes,categories,hours
0,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,2018-07-07 22:09:11,3.0,"If you decide to eat here, just be aware it is...",Turning Point of North Wales,1460 Bethlehem Pike,North Wales,PA,19454,40.210196,-75.223639,3.0,169,1,"{'NoiseLevel': 'u'average'', 'HasTV': 'False',...","Restaurants, Breakfast & Brunch, Food, Juice B...","{'Monday': '7:30-15:0', 'Tuesday': '7:30-15:0'..."
1,Iaee7y6zdSB3B-kRCo4z1w,XQfwVwDr-v0ZS3_CbbE5Xw,2017-05-13 17:06:55,2.0,This is the second time we tried turning point...,Turning Point of North Wales,1460 Bethlehem Pike,North Wales,PA,19454,40.210196,-75.223639,3.0,169,1,"{'NoiseLevel': 'u'average'', 'HasTV': 'False',...","Restaurants, Breakfast & Brunch, Food, Juice B...","{'Monday': '7:30-15:0', 'Tuesday': '7:30-15:0'..."
2,ejFxLGqQcWNLdNByJlIhnQ,XQfwVwDr-v0ZS3_CbbE5Xw,2017-08-08 00:58:18,4.0,The place is cute and the staff was very frien...,Turning Point of North Wales,1460 Bethlehem Pike,North Wales,PA,19454,40.210196,-75.223639,3.0,169,1,"{'NoiseLevel': 'u'average'', 'HasTV': 'False',...","Restaurants, Breakfast & Brunch, Food, Juice B...","{'Monday': '7:30-15:0', 'Tuesday': '7:30-15:0'..."
3,f7xa0p_1V9lx53iIGN5Sug,XQfwVwDr-v0ZS3_CbbE5Xw,2017-11-19 02:20:23,3.0,We came on a Saturday morning after waiting a ...,Turning Point of North Wales,1460 Bethlehem Pike,North Wales,PA,19454,40.210196,-75.223639,3.0,169,1,"{'NoiseLevel': 'u'average'', 'HasTV': 'False',...","Restaurants, Breakfast & Brunch, Food, Juice B...","{'Monday': '7:30-15:0', 'Tuesday': '7:30-15:0'..."
4,dCooFVCk8M1nVaQqcfTL3Q,XQfwVwDr-v0ZS3_CbbE5Xw,2017-09-09 17:49:47,2.0,"Mediocre at best. The decor is very nice, and ...",Turning Point of North Wales,1460 Bethlehem Pike,North Wales,PA,19454,40.210196,-75.223639,3.0,169,1,"{'NoiseLevel': 'u'average'', 'HasTV': 'False',...","Restaurants, Breakfast & Brunch, Food, Juice B...","{'Monday': '7:30-15:0', 'Tuesday': '7:30-15:0'..."


In [17]:
df2.to_csv('/content/gdrive/MyDrive/yelp_dataset/yelp_reviews_business_merge_restaurants.csv')

In [18]:
reviews_restaurants_only  = df2.iloc[: , :5]
reviews_restaurants_only = reviews_restaurants_only.rename(columns = {'stars_x':'stars'})

In [19]:
reviews_restaurants_only.head

<bound method NDFrame.head of                         user_id             business_id                 date  \
0        mh_-eMZ6K5RLWhZyISBhwA  XQfwVwDr-v0ZS3_CbbE5Xw  2018-07-07 22:09:11   
1        Iaee7y6zdSB3B-kRCo4z1w  XQfwVwDr-v0ZS3_CbbE5Xw  2017-05-13 17:06:55   
2        ejFxLGqQcWNLdNByJlIhnQ  XQfwVwDr-v0ZS3_CbbE5Xw  2017-08-08 00:58:18   
3        f7xa0p_1V9lx53iIGN5Sug  XQfwVwDr-v0ZS3_CbbE5Xw  2017-11-19 02:20:23   
4        dCooFVCk8M1nVaQqcfTL3Q  XQfwVwDr-v0ZS3_CbbE5Xw  2017-09-09 17:49:47   
...                         ...                     ...                  ...   
4724679  a4wr0eMRHjFqI66JoGpXog  Tj9FmBCHd84kjAE9vcoBnw  2018-04-06 01:19:30   
4724680  -MZSr4VErbuvzm8c5z2gbQ  Tj9FmBCHd84kjAE9vcoBnw  2018-04-01 04:36:21   
4724681  k5RiHhMsd7zT6gO-sa8CiQ  Tj9FmBCHd84kjAE9vcoBnw  2021-10-13 22:33:23   
4724682  cS5J5Lw1xECApPjmk6dyJg  Tj9FmBCHd84kjAE9vcoBnw  2021-11-03 18:05:23   
4724683  baGaUEUuXYQvTR9h4n9VKw  Tj9FmBCHd84kjAE9vcoBnw  2021-12-26 22:52:56   

         

In [20]:
reviews_restaurants_only.shape

(4724684, 5)

In [21]:
reviews_restaurants_only.to_csv('/content/gdrive/MyDrive/yelp_dataset/yelp_reviews_restaurants.csv')